In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0"

model_id = "/home/vapa/projects/iskra/models/asr/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch.float16, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=device,
)

/home/vapa/anaconda3/envs/iskra/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


 Hello everyone! How are you today?


In [ ]:
sample = "/home/vapa/projects/iskra/received_audio/test.wav"

result = pipe(sample)
print(result["text"])

In [1]:

from vllm import LLM, SamplingParams

llm_model_id = "/home/vapa/projects/iskra/models/llms/qwen2.5-7b-instruct"
llm = LLM(model=llm_model_id)

INFO 10-24 21:46:39 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/home/vapa/projects/iskra/models/llms/qwen2.5-7b-instruct', speculative_config=None, tokenizer='/home/vapa/projects/iskra/models/llms/qwen2.5-7b-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/vapa/projects/iskra/models/llms/qwen2.5-7b-in

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-24 21:46:42 model_runner.py:1067] Loading model weights took 14.2487 GB
INFO 10-24 21:46:45 gpu_executor.py:122] # GPU blocks: 2326, # CPU blocks: 4681
INFO 10-24 21:46:45 gpu_executor.py:126] Maximum concurrency for 32768 tokens per request: 1.14x
INFO 10-24 21:46:46 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-24 21:46:46 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-24 21:46:57 model_runner.py:1523] Graph capturing finished in 11 secs.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.13it/s, est. speed input: 17.07 toks/s, output: 34.15 toks/s]


TypeError: list indices must be integers or slices, not str

In [29]:
sampling_params = SamplingParams()
response = llm.generate(" Hello everyone! How are you today?", sampling_params=sampling_params)
request_output = response[0]
generated_text = request_output.outputs[0].text
print(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.03it/s, est. speed input: 16.29 toks/s, output: 32.58 toks/s]

 I hope you're having a great day!

Let's play a fun game with
